In [ ]:
import sys
sys.path.append('/home/mila/l/leo.gagnon/latent_control')

%load_ext autoreload
%autoreload 2
from tasks.dsm_diffusion import DSMDiffusion
import torch
import matplotlib.pyplot as plt
from data.diffusion import LatentDiffusionDataset, KnownEncoderDiffusionDataset
from torch2jax import j2t, t2j
import jax.numpy as jnp
import jax
from jax.scipy.special import rel_entr
from einops import repeat, rearrange
from models.encoder import KnownEncoder
from models.decoder import TransformerDecoder
from tqdm import tqdm
from tasks.metalearn import MetaLearningTask

/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/home/mila/l/leo.gagnon/late

In [38]:
from data.hmm import CompositionalHMMDataset, CompositionalHMMDatasetConfig
import jax.numpy as jnp
import jax.random as jr
from data.active import HMMEnv

# HMM

In [29]:
hmm = CompositionalHMMDataset(CompositionalHMMDatasetConfig(seed=0, has_actions=True))

In [46]:
hmm_ = CompositionalHMMDataset(CompositionalHMMDatasetConfig(seed=0, has_actions=False))
assert jnp.all(hmm.latent_transmat == hmm_.latent_transmat)
assert jnp.all(hmm_.latent_emissions == hmm_.latent_emissions)

In [30]:
X, Z = hmm.sample(1337, 200, jr.PRNGKey(0))

In [31]:
oracle_dict = hmm.bayesian_oracle(jnp.arange(len(hmm)), X)

In [32]:
hmm_post = jnp.exp(oracle_dict['log_alpha_post'][100])

In [33]:
assert hmm_post.max() == 1.0
assert hmm_post.argmax() == 1337

In [34]:
assert oracle_dict['z_post'][100].argmax() == Z[99]

In [35]:
oracle_dict_ = hmm.bayesian_oracle(jnp.arange(len(hmm)), X[100:], initial_messages=oracle_dict['messages'][99], log_alpha_prior=oracle_dict['log_alpha_post'][100])

In [36]:
assert jnp.allclose(oracle_dict['messages'][100:], oracle_dict_['messages'], atol=1e4)

In [47]:
hmm_env = HMMEnv(hmm, 0)

In [56]:
hmm_env.reset(1)
for i in range(199):
    hmm_env.step(jnp.array([[1.0,0.,0.,0.,0.,0.]]))

In [71]:
X = t2j(hmm_env.history[0, 0::2])

In [79]:
hmm_post = jnp.exp(hmm.bayesian_oracle(jnp.arange(len(hmm)), X)['log_alpha_post'])[100]

In [80]:
assert hmm_post.max() == 1.0
assert hmm_post.argmax() == hmm_env.indices[0].item()